<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading

**Chapter 10 &mdash; Trading Cryptocurrencies with Gemini**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

## Introduction

In [ ]:
import quandl as q
bn = q.get('BCHAIN/TOTBC') / 1e6  # in millions

In [ ]:
bn.plot(figsize=(10, 6))
plt.savefig('../../images/ch10/bitcoin_01.png');

In [ ]:
be = q.get('BCHAIN/MKPRU')

In [ ]:
be.plot(figsize=(10, 6))
plt.savefig('../../images/ch10/bitcoin_02.png');

In [ ]:
bm = q.get('BCHAIN/MKTCP') / 1e9  # in billions

In [ ]:
bm.plot(figsize=(10, 6))
plt.savefig('../../images/ch10/bitcoin_03.png');

## A Wrapper Class for the Gemini API

In [ ]:
import pygem
from pygem import config

In [ ]:
from importlib import reload

In [ ]:
reload(pygem)

In [ ]:
con = pygem.pygem(config['gemini']['key'],
                  config['gemini']['secret_key'],
                  debug=True)

In [ ]:
symbols = con.get_symbols()

In [ ]:
symbols

In [ ]:
con.get_ticker('btcusd')

In [ ]:
con.get_ticker('ethusd')

In [ ]:
ticker = con.get_ticker('ethbtc')

In [ ]:
ticker

In [ ]:
import datetime as dt

In [ ]:
dt.datetime.fromtimestamp(int(ticker['volume']['timestamp']) / 1e3)

## Retrieving Historical Data

In [ ]:
data = con.get_trades_history('btcusd', since=dt.datetime(2017, 1, 9, 11, 0, 0),
                             limit_trades=500, include_breaks=False)

In [ ]:
data.info()

In [ ]:
data.round(3).tail()

In [ ]:
data[['price', 'amount']].plot(secondary_y='amount', figsize=(10, 6))
plt.savefig('../../images/ch10/bitcoin_04.png');

In [ ]:
order = con.get_current_order_book('btcusd', limit_asks=5, limit_bids=5)

In [ ]:
import pprint

In [ ]:
pprint.pprint(order)

In [ ]:
import pandas as pd

In [ ]:
order = con.get_current_order_book('btcusd',
                                   limit_asks=500,
                                   limit_bids=500)

In [ ]:
bids = pd.DataFrame(order['bids'])

In [ ]:
bids['price'].astype(float).hist(bins=50, figsize=(10, 6))
plt.savefig('../../images/ch10/bitcoin_05.png');

## Placing and Managing Orders via the API

In [ ]:
new_order_result = con.new_order(symbol='btcusd', amount=0.001,
                                 price=1000, side='buy')

In [ ]:
new_order_result

In [ ]:
new_order_result_2 = con.new_order(symbol='btcusd', amount=0.0001,
                                 price=900, side='buy')

In [ ]:
new_order_result_2

In [ ]:
order_id = new_order_result_2['order_id']

In [ ]:
status = con.get_order_status(order_id)

In [ ]:
status

In [ ]:
cancel = con.cancel_order(order_id)

In [ ]:
cancel

## Most Recent Transaction History

In [ ]:
con = pygem.pygem(config['gemini']['key'],
                  config['gemini']['secret_key'],
                  debug=False)

In [ ]:
import time

In [ ]:
since = dt.datetime.now() - dt.timedelta(hours=3)

In [ ]:
trades = con.get_trades_history('ethusd', since=since,
                        limit_trades=5, include_breaks=False)
trades

In [ ]:
since = dt.datetime.now() - dt.timedelta(hours=1.1)

In [ ]:
for i in range(6):
    trades = con.get_trades_history('ethusd', since=since,
                                    limit_trades=500, include_breaks=False)
    print(40 * '-')
    print('timestamp %s' % dt.datetime.fromtimestamp(
                            float(trades['timestamp'].ix[-1])))
    print('#   trades: %d' % len(trades))
    print('ave. price: %f' % float(trades['price'].mean()))
    print('min  price: %f' % float(trades['price'].min()))
    print('max  price: %f' % float(trades['price'].max()))
    print('last price: %f' % float(trades['price'].ix[-1]))
    time.sleep(5)

## Implementing Trading Strategies in Real-Time 

In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame()

In [ ]:
messages = []

In [ ]:
def on_message(ws, message):
    global data
    global resam
    timestamp = dt.datetime.now()
    msg = json.loads(message)
    try:
        side = msg['events'][0]['side']
        price = float(msg['events'][0]['price'])

        if msg['type'] == 'update':
            
            print('%s | %s price = %f' % (timestamp, side, price))
            data = data.append(pd.DataFrame({side: price}, index=[timestamp]))
            messages.append(message)
            resam = data.resample('5s').last().ffill()
            resam['mid'] = resam.mean(axis=1)
    except:
        pass

In [ ]:
import websocket

In [ ]:
ws = websocket.WebSocketApp('wss://api.gemini.com/v1/marketdata/BTCUSD',
                            on_message=on_message)

In [ ]:
ws.run_forever(ping_interval=5)

In [ ]:
data.tail()

In [ ]:
resam.tail()

## Retrieving Account Information

In [ ]:
con = pygem.pygem(config['gemini']['key'],
                  config['gemini']['secret_key'],
                  debug=False)

In [ ]:
balances = con.get_available_balances()

In [ ]:
balances

In [ ]:
volumes = con.get_trade_volume()

In [ ]:
volumes

In [ ]:
orders = con.get_active_orders()

In [ ]:
orders

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

**Python Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>

**Listed Volatility and Variance Derivatives** |
<a href="http://lvvd.tpq.io" target="_blank">Listed VV Derivatives @ Wiley Finance</a>

**Python Training** |
<a href="http://training.tpq.io" target="_blank">Python for Finance University Certificate</a>